# 🧪 House Price Prediction with MLflow  
### A Mini Science Project using the California Housing Dataset

---

## 🔍 Abstract

In this project, we investigate how well a machine learning model can **predict house prices** using
features such as house age, average number of rooms, population, latitude, and longitude.

We will:

- Load the **California Housing** dataset
- Prepare the data into a structured `pandas` DataFrame
- Understand the **features** (inputs) and **target** (output)
- (Later) Train and tune a model, and track experiments using **MLflow**

This notebook covers the **data preparation and setup phase**, based on the lecture transcript.


## 🎯 Aim

To build a machine learning pipeline that predicts **median house value** using the
California Housing dataset and manages experiments with **MLflow**.

---

## ❓ Research Question

> Can we use features such as house age, average rooms, population, and geographic location  
> to accurately predict the **median house value** in California?

---

## 🧠 Hypothesis

If we provide a machine learning model with well-prepared features (e.g., average rooms, house age,
population, latitude, longitude), then:

- The model will learn patterns in the data
- It will be able to estimate the median house value with **reasonable accuracy**

We will later validate this using evaluation metrics such as **Mean Squared Error (MSE)**.


## 🔧 Materials & Tools

- **Programming Language**: Python
- **Notebook Environment**: Jupyter / VS Code Notebook
- **Libraries**:
  - `pandas` — for data manipulation
  - `sklearn.datasets` — to load the California Housing dataset
  - `sklearn.model_selection` — for train-test split and (later) hyperparameter tuning
  - `sklearn.ensemble` — Random Forest Regressor (later)
  - `sklearn.metrics` — Mean Squared Error (MSE) (later)
  - `mlflow` and `mlflow.sklearn` — to track experiments and models

In this notebook, we will start with:
- Installing/checking libraries
- Importing core dependencies
- Loading and preparing the dataset


In [ ]:
# import all libraries
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

## 🧪 Step 1: Environment Check

Before we start our experiment, we make sure:

- The correct **kernel** is selected (the environment where our packages are installed)
- Required libraries (`pandas`, `sklearn`, `mlflow`) are available

In Jupyter / VS Code:
- Look at the top-right corner and confirm the kernel matches your virtual environment.


In [2]:
# 🧪 Step 2: Data Collection

# Fetch the California Housing dataset
housing = fetch_california_housing()

# Inspect the keys of the dataset object
print("Keys in housing dataset object:", housing.keys())

print("\nFeature names:")
print(housing.feature_names)

print("\nData shape (rows, columns):", housing.data.shape)
print("Target shape:", housing.target.shape)


Keys in housing dataset object: dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

Feature names:
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']

Data shape (rows, columns): (20640, 8)
Target shape: (20640,)


In [3]:
# Take a closer look at the raw dataset structure
housing


{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]], shape=(20640, 8)),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': 

In [4]:
print("First 5 rows of raw feature data:\n", housing.data[:5])
print("\nFirst 5 target values (median house value):\n", housing.target[:5])


First 5 rows of raw feature data:
 [[ 8.32520000e+00  4.10000000e+01  6.98412698e+00  1.02380952e+00
   3.22000000e+02  2.55555556e+00  3.78800000e+01 -1.22230000e+02]
 [ 8.30140000e+00  2.10000000e+01  6.23813708e+00  9.71880492e-01
   2.40100000e+03  2.10984183e+00  3.78600000e+01 -1.22220000e+02]
 [ 7.25740000e+00  5.20000000e+01  8.28813559e+00  1.07344633e+00
   4.96000000e+02  2.80225989e+00  3.78500000e+01 -1.22240000e+02]
 [ 5.64310000e+00  5.20000000e+01  5.81735160e+00  1.07305936e+00
   5.58000000e+02  2.54794521e+00  3.78500000e+01 -1.22250000e+02]
 [ 3.84620000e+00  5.20000000e+01  6.28185328e+00  1.08108108e+00
   5.65000000e+02  2.18146718e+00  3.78500000e+01 -1.22250000e+02]]

First 5 target values (median house value):
 [4.526 3.585 3.521 3.413 3.422]


## 🔍 Observations: Dataset Structure

From the printed `housing` object, we observe:

- `data`: a 2D array of shape `(n_samples, n_features)` containing the input features
- `target`: a 1D array containing the **median house value** (our target variable)
- `feature_names`: names of each feature column, such as:
  - `MedInc` (median income)
  - `HouseAge`
  - `AveRooms`
  - `AveBedrms`
  - `Population`
  - `AveOccup`
  - `Latitude`
  - `Longitude`

These feature values will act as **inputs (X)** and the median house value (`target`) will act
as the **output (y)** for our model.


In [5]:
# 🧪 Step 3: Data Preparation

# Create a DataFrame from the feature data
data=pd.DataFrame(housing.data,columns=housing.feature_names)


# Add the target column: median house value (we'll call it 'price')
data["price"] = housing.target

# Inspect the top 10 rows
data.head(10)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
5,4.0368,52.0,4.761658,1.103627,413.0,2.139896,37.85,-122.25,2.697
6,3.6591,52.0,4.931907,0.951362,1094.0,2.128405,37.84,-122.25,2.992
7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25,2.414
8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26,2.267
9,3.6912,52.0,4.970588,0.990196,1551.0,2.172269,37.84,-122.25,2.611


## 🔍 Observations: Prepared Data

Now our dataset is in a **single DataFrame** called `data`:

- **Input Features (X)**:
  - `MedInc`, `HouseAge`, `AveRooms`, `AveBedrms`,
    `Population`, `AveOccup`, `Latitude`, `Longitude`

- **Target (y)**:
  - `price` → represents the **median house value**

We have successfully:

1. Loaded the raw dataset object using `fetch_california_housing()`
2. Converted the `data` array into a DataFrame with proper column names
3. Added a new column `price` using `housing.target`


# 🧪 Phase 2: Hyperparameter Tuning with Random Forest + MLflow

In this phase, we:

- Define **independent (X)** and **dependent (y)** variables
- Perform a **train–test split**
- Run **hyperparameter tuning** using `GridSearchCV`
- Log:
  - Best hyperparameters
  - Evaluation metrics
  - The trained model  
  into **MLflow**

We will use a **RandomForestRegressor** as our experimental model.


In [ ]:
from urllib.parse import urlparse  # for checking MLflow tracking store type

# Independent (X) and Dependent (y) features
X = data.drop(columns=["price"])
y = data["price"]

X.shape, y.shape


((20640, 8), (20640,))

In [7]:
# 🧪 Split data into training and test sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

from mlflow.models import infer_signature
# Infer signature for the model (input/output structure)
signature = infer_signature(X_train, y_train)

X_train.shape, X_test.shape

((16512, 8), (4128, 8))

In [8]:
# 🧪 Hyperparameter tuning using GridSearchCV

def hyperparameter_tuning(X_train, y_train, param_grid):
    rf = RandomForestRegressor(random_state=42)
    
    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=3,
        n_jobs=-1,
        verbose=2,
        scoring="neg_mean_squared_error"
    )
    
    grid_search.fit(X_train, y_train)
    return grid_search


In [9]:
# 🧪 Define the hyperparameter grid

param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}


In [11]:
# 🧪 Start MLflow experiment: hyperparameter tuning and logging

# Set MLflow tracking URI (your local MLflow server)
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Optional but nice: name the experiment
mlflow.set_experiment("house_price_california_rf_experiment")

with mlflow.start_run(run_name="rf_gridsearch"):
    # 1️⃣ Run hyperparameter tuning
    grid_search = hyperparameter_tuning(X_train, y_train, param_grid)
    
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    # 2️⃣ Evaluate on test set
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    
    # 3️⃣ Log parameters and metrics to MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    
    # 4️⃣ Decide how to log the model based on tracking store type
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    
    if tracking_url_type_store != "file":
        # e.g., remote tracking server with model registry
        mlflow.sklearn.log_model(
            best_model,
            "model",
            registered_model_name="Best_RandomForest_HousePrice"
        )
    else:
        # local file-based store (no registry)
        mlflow.sklearn.log_model(
            best_model,
            "model",
            signature=signature,
            input_example=X_train.iloc[:5]  # optional, but helpful
        )
    
    print(" Best Hyperparameters:", best_params)
    print(" Mean Squared Error (MSE):", mse)
    print(" Root Mean Squared Error (RMSE):", rmse)


2025/11/29 18:48:00 INFO mlflow.tracking.fluent: Experiment with name 'house_price_california_rf_experiment' does not exist. Creating a new experiment.
2025/11/29 18:48:00 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Fitting 3 folds for each of 24 candidates, totalling 72 fits


2025/11/29 18:52:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/29 18:53:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best_RandomForest_HousePrice'.
2025/11/29 18:53:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best_RandomForest_HousePrice, version 1
Created version '1' of model 'Best_RandomForest_HousePrice'.


✅ Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
✅ Mean Squared Error (MSE): 0.2539759249192041
✅ Root Mean Squared Error (RMSE): 0.5039602414072009
🏃 View run rf_gridsearch at: http://127.0.0.1:5000/#/experiments/941581754751804827/runs/27ef4d0073b7430b83f8106983b61a9d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/941581754751804827


## 🧾 Conclusion

In this experiment, we trained a **RandomForestRegressor** on the California Housing dataset and
used **GridSearchCV** together with **MLflow** to tune and track model performance.

The best-performing model used the following hyperparameters:

- `n_estimators`: **200**
- `max_depth`: **None** (trees grow until stopping rules)
- `min_samples_split`: **2**
- `min_samples_leaf`: **1**

With this configuration, the model achieved:

- **Mean Squared Error (MSE)**: `0.2539` (approx.)
- **Root Mean Squared Error (RMSE)**: `0.5040` (approx.)

Since the target variable represents the **median house value (in 100,000s of dollars)**, an
RMSE of about **0.50** corresponds to an average prediction error of roughly **$50,000**.
For a simple baseline model without heavy feature engineering, this is a reasonable starting
point, and it shows that the model has learned meaningful relationships between the input
features (e.g., `MedInc`, `HouseAge`, `AveRooms`, `Latitude`, `Longitude`) and the target
house prices.

---

## 🔭 Future Work

To improve the model and make the project more robust and production-ready, we can explore:

1. **Expanded Hyperparameter Search**
   - Try a wider range of values for:
     - `n_estimators` (e.g., 50–500)
     - `max_depth` (e.g., 3–30)
     - `min_samples_split`, `min_samples_leaf`
   - Use **RandomizedSearchCV** or Bayesian optimization for more efficient search.

2. **Model Comparison**
   - Train and compare other models:
     - Gradient Boosting, XGBoost, LightGBM, CatBoost
     - Linear models or regularized regressions as baselines
   - Log all results into **MLflow** and compare them in the UI.

3. **Feature Engineering**
   - Create new features (e.g., interaction terms, non-linear transforms).
   - Analyze feature importance and remove low-impact features.
   - Standardize or transform skewed variables if needed.

4. **Cross-Validation & Robust Evaluation**
   - Use **k-fold cross-validation** to get more stable performance estimates.
   - Track additional metrics such as **MAE** (Mean Absolute Error) and **R² score**.

5. **Model Deployment & Monitoring**
   - Register the best model in the **MLflow Model Registry**.
   - Deploy it as a REST API or service for real-time predictions.
   - Add monitoring for:
     - data drift,
     - performance degradation,
     - model versioning over time.

6. **Explainability**
   - Use tools like **SHAP** or **permutation importance** to understand
     which features drive house price predictions the most.

By iterating on these directions, we can transform this initial experiment into a more accurate,
explainable, and production-ready **house price prediction system**.
